# 整理数据

## 1. 拼接, 合并数据  


### 1.1 纵向拼接
```
pd.concat([df1, df2], ignore_index = True)
```

In [1]:
# setup
import pandas as pd
students_data = pd.DataFrame({'001': {"姓名":'小陈', '考试1':85, '考试2':95, '考试3':92}, '002': {"姓名":'小李', '考试1':91, '考试2':92, '考试3':94}, '003': {"姓名":'小王', '考试1':86, '考试2':81, '考试3':89}, '004': {"姓名":'小张', '考试1':79, '考试2':89, '考试3':95}, '005': {"姓名":'小赵', '考试1':96, '考试2':91, '考试3':91}, '006': {"姓名":'小周', '考试1':81, '考试2':89, '考试3':92} } )
students = pd.DataFrame(students_data).T
stu_1 = students.iloc[0:2]
stu_2 = students.iloc[2:]
stu_3 = stu_1.rename(columns={"考试3": "s"})
stu_4 = stu_2.copy()
stu_4.index = ['001', '002', '003', '004']

In [2]:
print(stu_1, '\n\n', stu_2)
print()
print(stu_3, '\n\n', stu_4)

     姓名 考试1 考试2 考试3
001  小陈  85  95  92
002  小李  91  92  94 

      姓名 考试1 考试2 考试3
003  小王  86  81  89
004  小张  79  89  95
005  小赵  96  91  91
006  小周  81  89  92

     姓名 考试1 考试2   s
001  小陈  85  95  92
002  小李  91  92  94 

      姓名 考试1 考试2 考试3
001  小王  86  81  89
002  小张  79  89  95
003  小赵  96  91  91
004  小周  81  89  92


In [3]:
print(pd.concat([stu_1, stu_2])) # 合并
print()
print(pd.concat([stu_1, stu_3])) # 列名不一样时, 匹配不上用NaN填充
print()
print(pd.concat([stu_1, stu_4], ignore_index=True ) ) # 纵向合并,忽略index, 

     姓名 考试1 考试2 考试3
001  小陈  85  95  92
002  小李  91  92  94
003  小王  86  81  89
004  小张  79  89  95
005  小赵  96  91  91
006  小周  81  89  92

     姓名 考试1 考试2  考试3    s
001  小陈  85  95   92  NaN
002  小李  91  92   94  NaN
001  小陈  85  95  NaN   92
002  小李  91  92  NaN   94

   姓名 考试1 考试2 考试3
0  小陈  85  95  92
1  小李  91  92  94
2  小王  86  81  89
3  小张  79  89  95
4  小赵  96  91  91
5  小周  81  89  92


### 1.2 横向合并
#### 列名合并
```
df = pd.merge(df1, df2, on = ['列名1', '列名2'], suffixes=["_stu_5", "_stu_6"] , how="inner") # 两个列都匹配才会合并 
```

`df = pd.merge(left=left_df, right=right_df, how='inner', left_on=’left_id', right_on='right_id’)`

| 关键字 | 含义（在 **两张表** 中） | 结果中保留的行 |
| ------ | ------------------------ | -------------- |
| `inner` | **交集**（只保留两张表中 **键值相同**的行） | 只出现在 **双方** 中的键 |
| `outer` | **并集**（把两张表的所有行都保留下来） | 所有键（不管是只在左表、只在右表，还是两者都有） |
| `left`  | **左外连接**（保留左表全部，右表只匹配的部分） | 左表的全部键 + 右表匹配的键 |
| `right` | **右外连接**（保留右表全部，左表只匹配的部分） | 右表的全部键 + 左表匹配的键 |

In [4]:
# 列名不统一, 客户编号 = 客户id, 日期 = 交易日期
# 注'客户编号', '日期' 和 '客户id', '交易日期' 都会保留在新列
pd.merge(df1, df2, left_on=['客户编号', '日期'], right_on=['客户id', '交易日期']) 


NameError: name 'df1' is not defined

In [5]:
stu_5 = students.iloc[0:3]
stu_6 = students.iloc[3:]
lst = stu_5['姓名'].copy()
stu_6['姓名'] = lst.to_numpy()
print(stu_5, '\n', stu_6)

     姓名 考试1 考试2 考试3
001  小陈  85  95  92
002  小李  91  92  94
003  小王  86  81  89 
      姓名 考试1 考试2 考试3
004  小陈  79  89  95
005  小李  96  91  91
006  小王  81  89  92


C:\Users\DBC\AppData\Local\Temp\ipykernel_17660\347772811.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stu_6['姓名'] = lst.to_numpy()


In [7]:
# 当按姓名合并, 但是有其他重名列时
new_pd = pd.merge(stu_5, stu_6, on = '姓名' , suffixes=["_stu_5", "_stu_6"] , how='inner')
'''
inner: 只保留两个都匹配的列
outer: 全部保留, 匹配不上NaN填充
left
'''
new_pd

,姓名,考试1_stu_5,考试2_stu_5,考试3_stu_5,考试1_stu_6,考试2_stu_6,考试3_stu_6
0,小陈,85,95,92,79,89,95
1,小李,91,92,94,96,91,91
2,小王,86,81,89,81,89,92


#### 索引合并
```
df1.join(df2, how='inner', lsuffix=df, rsuffix=df) # 当有重名index, 需指定suffix
```

## 2. 分组聚合  

### `df.groupby` 与 `pivot_table` 区别
- `df.groupby` :产生的值用列表示, 本质还是一维表, 用于继续数据分析
- `pivot_table`: 产生的值直接在单元格中,是二位表格, 用于制作报表

`df.groupby(['city']).groups.keys() `  

In [ ]:
import pandas as pd
import numpy as np

df_test = pd.DataFrame({
    'city'   : ['北京','北京','上海','上海','广州','广州','北京','上海'],
    'year'   : [2020,2021,2020,2021,2020,2021,2020,2021],
    'category': ['A','A','B','B','A','B','B','A'],
    'sales'  : [100,150,200,250,120,180,130,210],
    'profit' : [10,15,20,25,12,18,13,21]
})
df_test

,city,year,category,sales,profit
0,北京,2020,A,100,10
1,北京,2021,A,150,15
2,上海,2020,B,200,20
3,上海,2021,B,250,25
4,广州,2020,A,120,12
5,广州,2021,B,180,18
6,北京,2020,B,130,13
7,上海,2021,A,210,21


```
agg_dict = {
    "price": ["sum", "mean", price_range],
    "order_id": "count"
}

result = df.groupby("user_id").agg(agg_dict)
```

In [ ]:
# 按 city、year 分组，求 sales、profit 的均值, 双层index
gb = df_test.groupby(['city','year']).agg({'sales':'mean','profit':'mean'})
gb

sales  profit
city year               
上海   2020  200.0    20.0
     2021  230.0    23.0
北京   2020  115.0    11.5
     2021  150.0    15.0
广州   2020  120.0    12.0
     2021  180.0    18.0

#### `df.groupby('user_id')['price'].transform`

In [3]:
import pandas as pd
import numpy as np

df = pd.DataFrame({
    'user_id': [1, 1, 2, 2, 3],
    'price'  : [23.5, 45.0, np.nan, 40.0, 12.0]
})

# 1️⃣ 计算每个用户的 price 均值，并把均值写回每条记录
df['user_price_mean'] = df.groupby('user_id')['price'].transform('mean')
# → 对 user_id=1 的两行都得到 (23.5+45.0)/2 = 34.25

# 2️⃣ 用每个用户的均价填充缺失的 price
df['price_filled'] = df['price'].fillna(
    df.groupby('user_id')['price'].transform('mean')
)

# # 3️⃣ 对每个用户的 price 做 z‑score 标准化（返回同长 Series）
# df['price_z'] = df.groupby('user_id')['price'].transform(
#     lambda s: (s - s.mean()) / s.std()
# )

# # 4️⃣ 计算每行与本组最大值的差
# df['price_diff_max'] = df['price'] - df.groupby('user_id')['price'].transform('max')

df

,user_id,price,user_price_mean,price_filled
0,1,23.5,34.25,23.5
1,1,45.0,34.25,45.0
2,2,NaN,40.00,40.0
3,2,40.0,40.00,40.0
4,3,12.0,12.00,12.0


`pivot_city = pd.pivot_table(df,index=["city"])`

In [ ]:
# 直接得到交叉表
pt = pd.pivot_table(
    df_test,
    values='sales',          # 只聚合 sales
    index='city',            # 行维度
    columns='year',          # 列维度
    aggfunc='mean',          # 聚合方式
    fill_value=0             # 把缺失的格子填为 0
)
pt

year,2020,2021
city,,
上海,200,230
北京,115,150
广州,120,180



### `df.groupby("列名") # 并不返回有效数据, 需使用聚合函数`
<details>
<summary>聚合函数</summary> 

| 聚合函数 | 说明 | 示例 |
|----------|------|------|
| `sum()`   | 求和 | `df.groupby("部门")["工资"].sum()` |
| `mean()`  | 均值 | `df.groupby("部门")["工资"].mean()` |
| `median()`| 中位数 | `df.groupby("部门")["工资"].median()` |
| `max()`   | 最大值 | `df.groupby("部门")["工资"].max()` |
| `min()`   | 最小值 | `df.groupby("部门")["工资"].min()` |
| `count()` | 非空计数（相当于 `size()` 但不计入 NaN） | `df.groupby("部门")["工资"].count()` |
| `size()`  | 组内元素总数（包括 NaN） | `df.groupby("部门").size()` |
| `std()`   | 标准差 | `df.groupby("部门")["工资"].std()` |
| `var()`   | 方差 | `df.groupby("部门")["工资"].var()` |
| `first()` | 组内第一个值 | `df.groupby("部门")["工资"].first()` |
| `last()`  | 组内最后一个值 | `df.groupby("部门")["工资"].last()` |
| `nth(n)`  | 组内第 n 个元素（0‑based） | `df.groupby("部门")["工资"].nth(2)` |
| `agg(dict)`| 多列多聚合（自定义组合） | 见下方 **多列多聚合** 示例 |
| `describe()`| 汇总统计（count, mean, std, min, 25%, 50%, 75%, max） | `df.groupby("部门")["工资"].describe()` |
</details>

In [ ]:
# setup
import pandas as pd
import numpy as np

store_ids   = ["001", "002", "003"]                 # 分店编号
periods     = ["2025Q1", "2025Q2", "2025Q3"]        # 时间段
categories  = ["生鲜", "体闲"]                     # 商品类别

# -------------------------------------------------
# 3️⃣ 生成笛卡尔积（所有可能的组合）作为“基础表”
# -------------------------------------------------
base = pd.MultiIndex.from_product(
    [store_ids, periods, categories],
    names=["分店编号", "时间段", "商品类别"]
).to_frame(index=False)

# -------------------------------------------------
# 4️⃣ 为每条记录随机生成销售额、销售数量
# -------------------------------------------------
n = len(base)                                 # 记录总数（3×3×2 = 18）
base["销售额"]   = np.random.randint(5_000, 50_000, size=n)   # 单位：元
base["销售数量"] = np.random.randint(20, 500, size=n)        # 单位：件

df = base.copy()

In [ ]:
df

,分店编号,时间段,商品类别,销售额,销售数量
0,001,2025Q1,生鲜,48579,161
1,001,2025Q1,体闲,21598,415
2,001,2025Q2,生鲜,40236,251
3,001,2025Q2,体闲,21854,437
4,001,2025Q3,生鲜,25870,256
5,001,2025Q3,体闲,49041,60
6,002,2025Q1,生鲜,6369,374
7,002,2025Q1,体闲,43273,124
8,002,2025Q2,生鲜,28503,108
9,002,2025Q2,体闲,24067,39


In [ ]:
df.groupby('分店编号') # 并不是有效数据
df.groupby('分店编号')['销售额'].mean() #

分店编号
001    34529.666667
002    23738.500000
003    23574.166667
Name: 销售额, dtype: float64

In [ ]:
# 多层索引: 索引含 分店编号和时间段
grouped_df = df.groupby(['分店编号','时间段'])[['销售额', '销售数量']].mean()
# apply()
# df.groupby(['部门','员工'])[['工资', '绩效分']].apply(func)
grouped_df

销售额   销售数量
分店编号 时间段                   
001  2025Q1  35088.5  288.0
     2025Q2  31045.0  344.0
     2025Q3  37455.5  158.0
002  2025Q1  24821.0  249.0
     2025Q2  26285.0   73.5
     2025Q3  20109.5  333.5
003  2025Q1  12685.0  382.5
     2025Q2  17233.0  232.0
     2025Q3  40804.5  135.0

`grouped_df.reset_index()`  双层索引 -> 单索引

In [ ]:

#从多层索引 提取 
grouped_df.loc['001'].loc['2025Q1']
grouped_df.loc['001']

,销售额,销售数量
时间段,,
2025Q1,35088.5,288.0
2025Q2,31045.0,344.0
2025Q3,37455.5,158.0


In [ ]:
# 重置索引
grouped_df.reset_index()

,分店编号,时间段,销售额,销售数量
0,001,2025Q1,35088.5,288.0
1,001,2025Q2,31045.0,344.0
2,001,2025Q3,37455.5,158.0
3,002,2025Q1,24821.0,249.0
4,002,2025Q2,26285.0,73.5
5,002,2025Q3,20109.5,333.5
6,003,2025Q1,12685.0,382.5
7,003,2025Q2,17233.0,232.0
8,003,2025Q3,40804.5,135.0


### ``` pd.pivot_table()```

In [ ]:
# 把df的分店编号和时间段作为索引, 商品类别作为列, 计算销售总额
pd.pivot_table(df, index="商品类别", columns="分店编号", values='销售额') # 默认aggfunc=np.mean
pd.pivot_table(df, index=['分店编号', '时间段'], columns="商品类别", values='销售额', aggfunc=np.sum)

商品类别            体闲     生鲜
分店编号 时间段                 
001  2025Q1  21598  48579
     2025Q2  21854  40236
     2025Q3  49041  25870
002  2025Q1  43273   6369
     2025Q2  24067  28503
     2025Q3  31326   8893
003  2025Q1  18276   7094
     2025Q2  28027   6439
     2025Q3  43625  37984

### `pd.cut(df['列名'], bin_list)`

In [ ]:
# setup
import pandas as pd
import numpy as np


n = 30                         # 行数
emp_ids = [f"E{str(i).zfill(4)}" for i in range(1, n + 1)]
ages = np.random.randint(22, 61, size=n)

salary_raw = np.random.normal(loc=12000, scale=3000, size=n)
salary_raw = np.clip(salary_raw, 5000, 30000)   # 限制上下界
salaries = salary_raw.astype(int)

genders = np.random.choice(
    ["男", "女"],          # 取值
    size=n,                # 抽多少个
    p=[0.5, 0.5]           # 概率（这里默认 50/50）
)
# -------------------------------------------------
# 3️⃣ 创建 DataFrame
# -------------------------------------------------
df = pd.DataFrame({
    "员工号": emp_ids,
    "年龄": ages,
    "工资": salaries,
    "性别": genders
})
df.head()

,员工号,年龄,工资,性别
0,E0001,29,9117,女
1,E0002,52,5000,女
2,E0003,29,9783,女
3,E0004,28,12127,女
4,E0005,43,10995,男


In [ ]:
age_bins = [i for i in range(0, 61, 10)]
age_bins.append(120)
age_bins

age_labels = ['儿童', '青少年', '青年', '壮年', '中年', '中老年', '老年']

In [ ]:
# 新建年龄组列
df["年龄组"] = pd.cut(df['年龄'], age_bins, labels=age_labels )
df.groupby("年龄组")['工资'].mean()


年龄组
儿童              NaN
青少年             NaN
青年     11838.750000
壮年     12469.000000
中年     13180.300000
中老年    11513.142857
老年              NaN
Name: 工资, dtype: float64

### `df.query()` : 返回符合条件的观察值的索引   


In [ ]:
df.query("(性别 == '男') & (年龄 <= 50)") # <=>  
#df[(df['性别'] == '男') & (df['年龄'] <= 50 ) ]

,员工号,年龄,工资,性别,年龄组
4,E0005,43,10995,男,中年
5,E0006,45,11488,男,中年
6,E0007,40,9216,男,壮年
8,E0009,46,14585,男,中年
11,E0012,43,18641,男,中年
13,E0014,34,15608,男,壮年
15,E0016,50,11831,男,中年
20,E0021,49,13362,男,中年
21,E0022,30,9026,男,青年
26,E0027,27,15034,男,青年


# Advance: 多表关联 再删选


In [ ]:
# 横向
from functools import reduce

# 假设所有表的关联键都叫 `key`（可以提前 rename 成统一名称）
dfs = [students, scores, teachers, courses]   # 任意数量

# 统一键名（这里演示把不同键映射到统一列 `key`）
def rename_key(df, key_map):
    for old, new in key_map.items():
        if old in df.columns:
            df = df.rename(columns={old:new})
    return df

key_map = {
    'student_id':'key',
    'class'      :'key',
    'course_name':'key'      # 这里把课程表的 course_name 也改成 key
}
dfs = [rename_key(df, key_map) for df in dfs]

merged_all = reduce(lambda left, right: pd.merge(
                        left, right, on='key', how='outer',
                        suffixes=('_l','_r')), dfs)

print("\n=== 多表一次性合并（outer） ===")
print(merged_all.head())

pd.concat 纵向合并

In [ ]:
# 例：把每月的订单日志（结构相同）合并成一个大表
monthly_files = ["order_202301.csv","order_202302.csv","order_202303.csv"]
df_list = [pd.read_csv(f) for f in monthly_files]

all_orders = pd.concat(df_list, ignore_index=True)
print("\n=== 所有月份订单合并后行数:", all_orders.shape[0])